In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.tools as tls
from plotly.subplots import make_subplots

import importlib 


import aia_ops
from aia_ops.gaussian_fit_aia import gaussian_fit_aia
import error_function

In [2]:


def select_data(start_time,end_time,eve_line_name,aia_band_name):
    '''
    
    Input all the required parameters and return the selected aia_df and eve_df data
    The parameters are:
    start_time: the start time of the time range
    end_time: the end time of the time range
    eve_line_name: the line name of the EVE data
    aia_band_name: the band name of the AIA data, e.g. 304, 171, 193, 211, 131, 335

    '''

    # select aia_df data
    aia_df= pd.read_pickle('aia_ops//data//aia.pkl')
    # selected_aia_df = aia_df[(aia_df['date'] >= start_time) & (aia_df['date'] <= end_time+pd.Timedelta(hours=23,minutes=59,seconds=59))]
    selected_aia_df = aia_df[(aia_df['date'] >= start_time.date()) & (aia_df['date'] <= end_time.date()+pd.Timedelta(hours=23,minutes=59,seconds=59))]
    selected_aia_df.reset_index(drop=True, inplace=True)
    

    # select eve_df data
    selected_band=pd.read_pickle('eve_ops\data\selected_band.pkl')
    eve_daily_df=pd.read_pickle('eve_ops\data\daily_df\daily_df_'+eve_line_name+'.pkl')
    eve_daily_df = eve_daily_df[(eve_daily_df.index >= start_time) & (eve_daily_df.index <= end_time+pd.Timedelta(hours=23,minutes=59,seconds=59))]
    eve_daily_df.reset_index(inplace=True)
    eve_daily_df['date']=eve_daily_df['time'].dt.date

    # if (selected_aia_df.loc[0,'time'].date()==eve_daily_df.loc[0,'time'].date() and
    #     eve_daily_df.loc[0,'time'].date()==start_time.date() and
    #     eve_daily_df.loc[len(eve_daily_df)-1,'time'].date() == end_time.date() and
    #     selected_aia_df.loc[len(selected_aia_df)-1,'time'].date()==eve_daily_df.loc[len(eve_daily_df)-1,'time'].date()):

    if (selected_aia_df.loc[0,'date']==eve_daily_df.loc[0,'date'] and
        eve_daily_df.loc[0,'date']==start_time.date() and
        eve_daily_df.loc[len(eve_daily_df)-1,'date'] == end_time.date() and
        selected_aia_df.loc[len(selected_aia_df)-1,'date']==eve_daily_df.loc[len(eve_daily_df)-1,'date']):
        
        print('The selected aia_df and eve_df have the same date range.')
    else:
        raise ValueError('The selected aia_df and eve_df do not have the same date range, please check the code on date selection.')
    


    # get the wavelength list in the simulation using AIA images
    eve_selected_band=pd.read_pickle('eve_ops\data\selected_band.pkl')
    eve_wavelength_full=np.load('eve_ops/data/wavelength_full.npz')['wavelength_full']
                                
    # print('eve_selected_band:',eve_selected_band)
    # print('eve_wavelength_full:',eve_wavelength_full)

    wavelength_list_aia_index = eve_selected_band[eve_selected_band['Line Name'] == eve_line_name]['Wavelength Index'].values[0]
    accurate_wavelength = eve_selected_band[eve_selected_band['Line Name'] == eve_line_name]['Accurate Wavelength'].values[0]/10    # unit: nm
    wavelength_list_aia = eve_wavelength_full[wavelength_list_aia_index]-accurate_wavelength



    aia_initial_guess=[1.4e9,0,0.04]

    return selected_aia_df,eve_daily_df,accurate_wavelength,wavelength_list_aia,aia_initial_guess


In [11]:
cruciform_df = pd.read_pickle('aia_ops/data/catalog/cruciform_df.pkl')
# start_time=cruciform_df.loc[10,'Date'] # 2012-07-10 00:00:00
# end_time=cruciform_df.loc[11,'Date'] # 2012-10-10 00:00:00

start_time=cruciform_df.loc[5,'Date'] # 2012-07-10 00:00:00
end_time=cruciform_df.loc[15,'Date'] # 2012-10-10 00:00:00


eve_line_name='He II 304'

aia_band_name='304'

print('All the input for the fit:')
print('start_time:', start_time)
print('end_time:', end_time)
print('eve_line_name:', eve_line_name)
print('aia_band_name:', aia_band_name)

All the input for the fit:
start_time: 2011-07-13 00:00:00
end_time: 2014-01-15 00:00:00
eve_line_name: He II 304
aia_band_name: 304


In [17]:
# coeff=[886, 0, 0, .91, -0.0097]
coeff=[-3148.8558196722797, -8.655290403288173, 2.873857139833321, 3.0413785018967996, 0.010358837503063881]

In [18]:
selected_aia_df,selected_eve_df,accurate_wavelength,wavelength_list_aia,aia_initial_guess=select_data(start_time,end_time,eve_line_name,aia_band_name)

The selected aia_df and eve_df have the same date range.


In [19]:
combine_df=error_function.get_df_to_plot(coeff,selected_aia_df,wavelength_list_aia,aia_initial_guess,accurate_wavelength, selected_eve_df)

In [20]:
combine_df

,date,aia_simu,simu_amplitude,simu_mean,simu_stddev,time,median_of_mean,median_of_stddev,median_of_amplitude,stddev_of_mean,stddev_of_stddev,stddev_of_amplitude,available_data,median_of_doppler
0,2011-07-13,"[74595573.1376712, 188962157.4145683, 39336550...",1.079939e+09,30.373073,0.044315,2011-07-13,30.373371,0.029955,0.006464,0.000329,0.000097,0.000051,6924,-45686.345154
1,2011-07-14,"[74907432.44289914, 190012130.18085837, 396025...",1.089868e+09,30.373111,0.044289,2011-07-14,30.373305,0.030012,0.006545,0.000389,0.000112,0.000039,8440,-46336.671558
2,2011-07-15,"[76120650.73558757, 193028548.46251392, 402236...",1.107171e+09,30.373129,0.044296,2011-07-15,30.373216,0.030050,0.006609,0.000399,0.000108,0.000040,8584,-47208.555040
3,2011-07-16,"[76401822.68459359, 193987474.79747906, 404752...",1.118246e+09,30.373226,0.044282,2011-07-16,30.373087,0.030031,0.006660,0.000417,0.000076,0.000039,8583,-48485.271380
4,2011-07-17,"[77348626.54423787, 196369870.15800947, 409857...",1.137294e+09,30.373425,0.044323,2011-07-17,30.372926,0.030073,0.006806,0.000421,0.000088,0.000048,8515,-50071.877582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,2014-01-11,"[96865744.27270883, 246919353.97412497, 517783...",1.449179e+09,30.373440,0.044151,2014-01-11,30.371442,0.031352,0.007446,0.000215,0.000139,0.000072,8581,-64718.044639
913,2014-01-12,"[95364445.72309388, 242847123.74922353, 509131...",1.429934e+09,30.373598,0.044180,2014-01-12,30.371375,0.031377,0.007303,0.000278,0.000111,0.000086,7791,-65383.244418
914,2014-01-13,"[93033465.53994197, 236057693.39287472, 493424...",1.380262e+09,30.373594,0.044261,2014-01-13,30.371081,0.031345,0.007145,0.000227,0.000108,0.000057,8220,-68285.978057
915,2014-01-14,"[92357131.23065324, 233650810.97932658, 486967...",1.351886e+09,30.373418,0.044304,2014-01-14,30.370937,0.031302,0.007095,0.000248,0.000148,0.000088,8221,-69705.394341


In [21]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=.011)

valid_day=np.where(combine_df['available_data']>=np.nanmean(combine_df['available_data'])*0.2,1,np.nan)
x = combine_df['date']
y_median = combine_df['median_of_mean']*valid_day
y_stddev = combine_df['stddev_of_mean']
fill_x = np.concatenate([x, x[::-1]])
fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])

# EVE: Plot the mean line
fig.add_trace(go.Scatter(x=x, y=y_median,
                                # showlegend=False, 
                        line=dict(color='rgb(0,128,255)', ),
                        name='EVE observation') ,
                row=3, col=1,)
# Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
fig.add_trace(go.Scatter(x=fill_x, y=fill_y, mode='lines', 
                                fill='tozerox', 
                                fillcolor='rgba(0,176,246,0.2)',
                                line=dict(color='rgba(127,255,0,0.2)'),
                                showlegend=False, 
                                name='3 Std. Dev.'),
                row=3, col=1)
#AIA
fig.add_trace(go.Scatter(x=combine_df['date'], y=combine_df['simu_mean'],
                            # showlegend=False, 
                    line=dict(color='rgb(255,128,53)'),
                                name='AIA simulation') ,
            row=3, col=1,)

fig.update_yaxes(title_text="Mean (nm)",range=[30.3705, 30.3755],row=3, col=1)
# Show the plot



# Create a figure for stddev
x = combine_df['date']
y_median = combine_df['median_of_stddev']*valid_day
y_stddev = combine_df['stddev_of_stddev']
fill_x = np.concatenate([x, x[::-1]])
fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])

fig.add_trace(go.Scatter(x=x, y=y_median,
                            showlegend=False,
                            line=dict(color='rgb(0,128,255)'),),
                        row=2, col=1)
# Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
fig.add_trace(go.Scatter(x=fill_x, y=fill_y, mode='lines', 
                                fill='tozerox', 
                                fillcolor='rgba(0,176,246,0.2)',
                                line=dict(color='rgba(127,255,0,0.2)'),
                                showlegend=False,
                                name='3 Std. Dev.'),
                    row=2, col=1)
#AIA
fig.add_trace(go.Scatter(x=combine_df['date'], y=combine_df['simu_stddev'],
                            showlegend=False, 
                    line=dict(color='rgb(255,128,53)', )) ,
            row=2, col=1,)
fig.update_yaxes(title_text="Stddev (nm)",range=[0.026, 0.032],row=2, col=1)





# # Create a figure for amplitude
x = combine_df['date']
y_median = combine_df['median_of_amplitude']*valid_day
y_stddev = combine_df['stddev_of_amplitude']
fill_x = np.concatenate([x, x[::-1]])
fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])


fig.add_trace(go.Scatter(x=combine_df['date'], y=y_median/np.nanmean(y_median),
                                            showlegend=False,
                        line=dict(color='rgb(0,128,255)'),),
                            row=1, col=1)
# Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
fig.add_trace(go.Scatter(x=fill_x, y=fill_y/np.nanmean(y_median), mode='lines', 
                                fill='tozerox', 
                                fillcolor='rgba(0,176,246,0.2)',
                                line=dict(color='rgba(127,255,0,0.2)'),
                                showlegend=False, 
                                name='3 Std. Dev.'),
                        row=1, col=1)
#AIA
fig.add_trace(go.Scatter(x=combine_df['date'], y=combine_df['simu_amplitude']/np.nanmean(combine_df['simu_amplitude']),
                            showlegend=False, 
                    line=dict(color='rgb(255,128,53)', )) ,
            row=1, col=1,)
fig.update_yaxes(title_text='relative amplitude ',
                    # range=[0.6, 1.4],
                    row=1, col=1)



# total fig adjustment
fig.update_layout(width=1200,height=1500,
                        title_text=f"EVE obs vs. AIA simulation a={coeff[0]:.2f}, b={coeff[1]:.2f}, c={coeff[2]:.2f}, d={coeff[3]:.2f}, e={coeff[4]:.2f}",
                        legend=dict(x=1, y=0.9)) 
fig.update_xaxes(title_text="Time",row=3, col=1)

# #save & display
# if(output_fig):
#     fig.write_html(output_fig)
# fig.show() 

fig.write_html('output//para_trial_plotly//gradient_descent_result.html')

In [ ]:
def plotly_2d_combine(selected_df,simu_df,combine_df,output_fig):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=.011)

    valid_day=np.where(selected_df['available_data']>=np.nanmean(selected_df['available_data'])*0.2,1,np.nan)
    x = selected_df.index
    y_median = selected_df['median_of_mean']*valid_day
    y_stddev = selected_df['stddev_of_mean']
    fill_x = np.concatenate([x, x[::-1]])
    fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])

    # EVE: Plot the mean line
    fig.add_trace(go.Scatter(x=x, y=y_median,
                                  # showlegend=False, 
                            line=dict(color='rgb(0,128,255)', ),
                            name='EVE') ,
                  row=3, col=1,)
    # Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
    fig.add_trace(go.Scatter(x=fill_x, y=fill_y, mode='lines', 
                                  fill='tozerox', 
                                  fillcolor='rgba(0,176,246,0.2)',
                                  line=dict(color='rgba(127,255,0,0.2)'),
                                  showlegend=False, 
                                  name='3 Std. Dev.'),
                 row=3, col=1)
    #AIA
    fig.add_trace(go.Scatter(x=simu_df.index, y=simu_df['mean'],
                              # showlegend=False, 
                        line=dict(color='rgb(255,128,53)'),
                                 name='AIA') ,
              row=3, col=1,)
    
    fig.update_yaxes(title_text="Mean (nm)",range=[30.3705, 30.3755],row=3, col=1)
    # Show the plot



    # Create a figure for stddev
    x = selected_df.index
    y_median = selected_df['median_of_stddev']*valid_day
    y_stddev = selected_df['stddev_of_stddev']
    fill_x = np.concatenate([x, x[::-1]])
    fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])

    fig.add_trace(go.Scatter(x=x, y=y_median,
                                showlegend=False,
                             line=dict(color='rgb(0,128,255)'),),
                          row=2, col=1)
    # Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
    fig.add_trace(go.Scatter(x=fill_x, y=fill_y, mode='lines', 
                                  fill='tozerox', 
                                  fillcolor='rgba(0,176,246,0.2)',
                                  line=dict(color='rgba(127,255,0,0.2)'),
                                  showlegend=False,
                                  name='3 Std. Dev.'),
                        row=2, col=1)
    #AIA
    fig.add_trace(go.Scatter(x=simu_df.index, y=simu_df['stddev'],
                              showlegend=False, 
                        line=dict(color='rgb(255,128,53)', )) ,
              row=2, col=1,)
    fig.update_yaxes(title_text="Stddev (nm)",range=[0.026, 0.032],row=2, col=1)





    # # Create a figure for amplitude
    x = selected_df.index
    y_median = selected_df['median_of_amplitude']*valid_day
    y_stddev = selected_df['stddev_of_amplitude']
    fill_x = np.concatenate([x, x[::-1]])
    fill_y = np.concatenate([y_median + 3 * y_stddev, (y_median - 3 * y_stddev)[::-1]])


    fig.add_trace(go.Scatter(x=selected_df.index, y=y_median/np.nanmean(y_median),
                                             showlegend=False,
                            line=dict(color='rgb(0,128,255)'),),
                             row=1, col=1)
    # Plot the shaded area between mean + 3 std deviations and mean - 3 std deviations
    fig.add_trace(go.Scatter(x=fill_x, y=fill_y/np.nanmean(y_median), mode='lines', 
                                  fill='tozerox', 
                                  fillcolor='rgba(0,176,246,0.2)',
                                  line=dict(color='rgba(127,255,0,0.2)'),
                                  showlegend=False, 
                                  name='3 Std. Dev.'),
                           row=1, col=1)
    #AIA
    fig.add_trace(go.Scatter(x=simu_df.index, y=simu_df['amplitude']/np.nanmean(simu_df['amplitude']),
                              showlegend=False, 
                        line=dict(color='rgb(255,128,53)', )) ,
              row=1, col=1,)
    fig.update_yaxes(title_text='relative amplitude ',
                     # range=[0.6, 1.4],
                     row=1, col=1)



    # total fig adjustment
    fig.update_layout(width=1200,height=1500,
                          title_text=f"EVE AIA",
                         legend=dict(x=1, y=0.9)) 
    fig.update_xaxes(title_text="Time",row=3, col=1)

    #save & display
    if(output_fig):
        fig.write_html(output_fig)
    fig.show() 